In [ ]:
%load_ext autoreload
%autoreload 2

from cmipper import functions_creche, utils, config, parallelised_download_and_process, main
import xarray as xa

In [ ]:
xa.open_dataset("/maps/rt582/cmipper/data/env_vars/cmip6/EC-Earth3P-HR/r1i1p2f1/og_grid/tos/tos_uncropped_sfc_tp_200401-200412.nc")

In [ ]:
xa.open_dataset("/maps-priv/maps/rt582/cmipper/data/env_vars/cmip6/EC-Earth3P-HR/r1i1p2f1/newtest/og_grid/mlotst/mlotst_uncropped_sfc_tp_199301-199312.nc")

In [ ]:
parallelised_download_and_process.delete_corrupt_files("EC-Earth3P-HR", "r1i1p2f1")

In [ ]:
main.main()